In [1]:
import lightgbm as lgbm
import pandas as pd
import numpy as np

In [2]:
## data load
df_origin = pd.read_csv("./data/train_idall_et10_row60_col3273_Delunique.csv")

df_x = df_origin.copy()

In [3]:
test_origin = pd.read_csv("./data/test_idall_et10_row60_col3273_Delunique.csv")

test_x = test_origin.copy()
x_test = test_x.iloc[:,1:]
x_test.shape

(36000, 3273)

In [ ]:
submission = pd.DataFrame(data=preds)
submission.index = index
submission.index.name = 'id'
submission = submission.sort_index()
submission = submission.groupby('id').mean()
submission.to_csv('./data/submission_LGBM_10_60초_2.csv', index=True)
submission.head()

In [4]:
test_x.rename(columns = {'Unnamed: 0' : 'index'}, inplace = True)
index = test_x["index"]
test_x.head()


,index,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,...,V5087,V5088,V5089,V5090,V5115,V5116,V5117,V5118,V5119,V5120
0,1000,30.465741,8.618514,8.705075,8.730912,8.699214,181.327530,201.889419,2.393806e-19,0.0,...,-0.229694,-0.172642,-0.181698,43.203640,60.0,0.0,0.0,0.000004,85.4,0.0
1,1000,30.477302,8.642689,8.713423,8.732450,8.694666,203.347675,155.790045,-1.808861e-19,0.0,...,-0.232155,-0.177659,-0.164828,43.195677,60.0,0.0,0.0,0.000005,85.4,0.0
2,1000,30.478336,8.675928,8.729837,8.672877,8.710215,196.673652,227.039249,6.236627e-19,0.0,...,-0.249112,-0.193733,-0.170861,43.192395,60.0,0.0,0.0,0.000016,85.4,0.0
3,1000,30.462904,8.733765,8.706455,8.691974,8.696285,194.365551,167.436935,2.845012e-20,0.0,...,-0.240052,-0.166091,-0.164244,43.195476,60.0,0.0,0.0,-0.000010,85.4,0.0
4,1000,30.483675,8.807382,8.680733,8.713651,8.664766,205.369347,154.245975,-2.085638e-19,0.0,...,-0.230030,-0.228277,-0.139462,43.193896,60.0,0.0,0.0,-0.000008,85.4,0.0


In [5]:

x_train = df_x.iloc[:,1:-1]
x_train.head()
label = df_x["label"]
x_train.head()

,V0000,V0001,V0002,V0003,V0004,V0005,V0006,V0007,V0008,V0009,...,V5087,V5088,V5089,V5090,V5115,V5116,V5117,V5118,V5119,V5120
0,30.469852,8.731628,8.702328,8.762145,8.709091,200.426266,178.552403,-2.938919e-19,0.0,9.633728e-05,...,-0.252310,-0.210976,-0.148770,43.195434,60.0,0.0,0.0,0.000008,85.4,0.0
1,30.456702,8.640097,8.681890,8.702515,8.720491,191.522955,214.903659,6.805364e-19,0.0,-1.211133e-03,...,-0.236125,-0.233830,-0.165802,43.203458,60.0,0.0,0.0,-0.000002,85.4,0.0
2,30.493822,8.669871,8.704550,8.687762,8.718178,161.361161,179.465434,1.746378e-20,0.0,2.422847e-03,...,-0.220420,-0.151361,-0.176624,43.205211,60.0,0.0,0.0,-0.000020,85.4,0.0
3,30.480267,8.754620,8.688407,8.729588,8.723822,204.495185,188.033847,-5.755259e-19,0.0,9.121588e-07,...,-0.226720,-0.226633,-0.169439,43.197066,60.0,0.0,0.0,-0.000007,85.4,0.0
4,30.448453,8.706687,8.700838,8.732337,8.710176,197.503313,188.625723,1.332854e-19,0.0,1.806481e-03,...,-0.235123,-0.271625,-0.165099,43.207469,60.0,0.0,0.0,-0.000015,85.4,0.0


In [6]:
# train_data = lgb.Dataset('x_train')d
d_train = lgbm.Dataset(x_train, label=label)
d_train

In [ ]:
# multiclassova

params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':198,
    'metric': 'multi_logloss',
          'is_training_metric': True,
    'learning_rate': 0.005,
          'metric_freq': 10,
    'num_leaves': 50,
         'max_depth':-1,
          'force_col_wise':True,
'feature_fraction': 0.9,
'bagging_fraction': 0.8,
'bagging_freq': 10,
'verbose': 1}

#     # 'feature_fraction': 0.4,
#     # 'bagging_fraction': 0.6,
#     # 'bagging_freq': 17

In [ ]:
%%time
lgb_cv = lgbm.cv(params, d_train, num_boost_round=600, nfold=3, shuffle=True, stratified=True, verbose_eval=100, early_stopping_rounds=30)

nround = lgb_cv['multi_logloss-mean'].index(np.min(lgb_cv['multi_logloss-mean']))
print(nround)

model = lgbm.train(params, d_train, num_boost_round=nround)

[100]	cv_agg's multi_logloss: 1.24815 + 0.00727066
[200]	cv_agg's multi_logloss: 0.783995 + 0.00687085
[300]	cv_agg's multi_logloss: 0.603658 + 0.00621398
[400]	cv_agg's multi_logloss: 0.524338 + 0.00601926
[500]	cv_agg's multi_logloss: 0.485509 + 0.00615232
[600]	cv_agg's multi_logloss: 0.464357 + 0.00649451
599


In [ ]:
preds = model.predict(x_test)

In [ ]:
preds.shape

In [ ]:
preds1 = model.predict(x_test,num_iteration=lgbm.best_iteration)

In [ ]:
num_iteration=bst.best_iteration

sklearn.tree.DecisionTreeClassifier

sklearn.tree.ExtraTreeClassifier x

sklearn.ensemble.ExtraTreesClassifier x

sklearn.neighbors.KNeighborsClassifier x

sklearn.neural_network.MLPClassifier x

sklearn.neighbors.RadiusNeighborsClassifier 

sklearn.ensemble.RandomForestClassifier x
 
sklearn.linear_model.RidgeClassifierCV x

멀티 클래스 멀티 출력 지원 :

sklearn.tree.DecisionTreeClassifier

sklearn.tree.ExtraTreeClassifier

sklearn.ensemble.ExtraTreesClassifier

sklearn.neighbors.KNeighborsClassifier

sklearn.neighbors.RadiusNeighborsClassifier

sklearn.ensemble.RandomForestClassifier



In [ ]:
params = {
'task': 'train',
'boosting_type': 'gbdt',
'objective': 'multiclass',
'num_class':198,
'metric': 'mulit_logloss',
'is_training_metric': True,
'metric_freq': 5,
'num_leaves': 35,
'learning_rate': 0.01,
'max_depth':17,
'feature_fraction': 0.9,
'bagging_fraction': 0.8,
'bagging_freq': 5,
'verbose': 1,
'device' : 'cpu',
'gpu_platform_id' : 0,
'gpu_device_id' : 0
}

# gbm = lgb.train(
# params,
# lgb_train,
# num_boost_round=20,
# valid_sets=lgb_eval,
# early_stopping_rounds=5)

In [7]:
# first model param

params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':198,
    'metric': 'multi_logloss',
          'is_training_metric': True,
    'learning_rate': 0.01,
          'metric_freq': 5,
    'num_leaves': 34,
         'max_depth':17,
'feature_fraction': 0.8,
'bagging_fraction': 0.7,
'bagging_freq': 14,
'verbose': 1}